In [1]:
# =========================================================
# 1. Imports
# =========================================================
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_classif

from imblearn.over_sampling import SMOTE

import matplotlib.pyplot as plt

In [2]:
# =========================================================
# 2. Load dataset
# =========================================================
df = pd.read_csv("data/online_shoppers_intention.csv")
print("Dataset loaded:", df.shape)

Dataset loaded: (12330, 18)


In [ ]:
# =========================================================
# 3. Convert boolean variables to numeric
# =========================================================
df['Revenue'] = df['Revenue'].astype(int)
df['Weekend'] = df['Weekend'].astype(int)


# =========================================================
# 4. Group rare categories in numeric-categorical features
# =========================================================
numeric_categorical = ['OperatingSystems', 'Browser', 'Region', 'TrafficType']

def group_rare(series, threshold=50):
    freq = series.value_counts()
    return series.apply(lambda x: x if freq[x] > threshold else "Other")

for col in numeric_categorical:
    df[col] = df[col].astype(str)
    df[col] = group_rare(df[col])


# =========================================================
# 5. Define categorical & numerical sets
# =========================================================
categorical_nominal = ['Month', 'VisitorType'] + numeric_categorical
numerical_features_original = [
    'Administrative', 'Administrative_Duration',
    'Informational', 'Informational_Duration',
    'ProductRelated', 'ProductRelated_Duration',
    'BounceRates', 'ExitRates', 'PageValues',
    'SpecialDay'
]


# =========================================================
# 6. Train-test split (BEFORE encoding!)
# =========================================================
X = df.drop('Revenue', axis=1)
y = df['Revenue']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Train:", X_train.shape, " Test:", X_test.shape)


# =========================================================
# 7. Log-transform duration columns
# =========================================================
for col in ['Administrative_Duration', 'Informational_Duration', 'ProductRelated_Duration']:
    X_train[col] = np.log1p(X_train[col])
    X_test[col] = np.log1p(X_test[col])


# =========================================================
# 8. Fit OneHotEncoder on TRAIN split
# =========================================================
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
encoder.fit(X_train[categorical_nominal])

# Transform
X_train_cat = encoder.transform(X_train[categorical_nominal])
X_test_cat  = encoder.transform(X_test[categorical_nominal])

X_train = X_train.drop(columns=categorical_nominal)
X_test = X_test.drop(columns=categorical_nominal)

# Get feature names (optional)
encoded_cols = encoder.get_feature_names_out(categorical_nominal)


# =========================================================
# Define numerical columns AUTOMATICALLY
# =========================================================
numerical_features = X_train.select_dtypes(include=[np.number]).columns

X_train_num = X_train[numerical_features].reset_index(drop=True)
X_test_num  = X_test[numerical_features].reset_index(drop=True)


X_train_full = np.hstack([X_train_num.values, X_train_cat])
X_test_full  = np.hstack([X_test_num.values,  X_test_cat])



# =========================================================
# 10. Scale numerical + encoded features
# =========================================================
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train_full)
X_test_scaled  = scaler.transform(X_test_full)


# =========================================================
# 11. PCA (for visualization only)
# =========================================================
pca = PCA(n_components=2)
pca.fit(X_train_scaled)

X_train_pca = pca.transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# =========================================================
# 12. SMOTE (APPLY ONLY ON SCALED DATA, ONLY FOR TRAINING MODELS THAT NEED IT)
# =========================================================
# Example: only for tree models, not for LogisticRegression

smote = SMOTE(random_state=0)
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

print("After SMOTE:", X_train_smote.shape)


After encoding: (12330, 56)
